### LDA

Approach
- Only selected noun, adj, verb, adverb in the corpus
- Got the lemma of those words
- Removed stopwords including accents (deacc) like Polish etc and broke sentence down into individual words
- BOW dictionary - unique word and frequency in the doc


In [3]:
#https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/#1introduction
import numpy as np
import json
import glob

#Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

#spacy
import spacy
from nltk.corpus import stopwords

#vis
import pyLDAvis
import pyLDAvis.gensim

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

Prepare the data

In [4]:
def load_data(file):
    with open (file, "r", encoding="utf-8") as f:
        data = json.load(f) 
    return (data)

def write_data(file, data):
    with open (file, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=4)

In [5]:
stopwords = stopwords.words("english")

In [6]:
print (stopwords)

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she

In [7]:
data = load_data("data/ushmm_dn.json")["texts"]

print (data[0][0:90])

 My name David Kochalski. I was born in a small town called , and I was born May 5, 1928. 


You can use nltk or spacy for lemmatization, here we will use spacy

Consider only noun, adjective, verb and adverb

In [8]:
def lemmatization(texts, allowed_postags=["NOUN", "ADJ", "VERB", "ADV"]):
    nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])
    texts_out = []
    for text in texts:
        doc = nlp(text)
        new_text = []
        for token in doc:
            if token.pos_ in allowed_postags:
                new_text.append(token.lemma_)
        final = " ".join(new_text)
        texts_out.append(final)
    return (texts_out)


lemmatized_texts = lemmatization(data)
print (lemmatized_texts[0][0:90])

name bear small town call bear very hard work child father mother small mill flour buckwhe


Remove stopwords and get individual words. Gensim model needs individual words

In [9]:
def gen_words(texts):
    final = []
    for text in texts:
        new = gensim.utils.simple_preprocess(text, deacc=True)
        final.append(new)
    return (final)

data_words = gen_words(lemmatized_texts)

print (data_words[0][0:20])

['name', 'bear', 'small', 'town', 'call', 'bear', 'very', 'hard', 'work', 'child', 'father', 'mother', 'small', 'mill', 'flour', 'buckwheat', 'prosperous', 'comfortable', 'go', 'school']


Create bag of words dictionary. It will have unique words and the frequency in the corpus

In [10]:
id2word = corpora.Dictionary(data_words)

corpus = []
for text in data_words:
    new = id2word.doc2bow(text)
    corpus.append(new)

print (corpus[0][0:20])

word = id2word[[0][:1][0]]
print (word)

[(0, 2), (1, 10), (2, 1), (3, 2), (4, 1), (5, 1), (6, 2), (7, 3), (8, 1), (9, 12), (10, 1), (11, 8), (12, 1), (13, 2), (14, 1), (15, 3), (16, 2), (17, 1), (18, 1), (19, 1)]
able


In [20]:
id2word[[5][0:1][0]]

'activity'

In [21]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=30,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha="auto")

### Visualize the data

We see some problems:
1. We asked for 30 clusters but we can see only 10
2. The words showing up in the cluster seem like stopwords which should be removed, they don't explain any particular topic so we need to work on custom stopword removal to get better topics

In [22]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word, mds="mmds", R=30)
vis

c:\Users\Kimberly\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\manifold\_mds.py:677: FutureWarning: The default value of `n_init` will change from 4 to 1 in 1.9.
  warnings.warn(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
22     0.052503 -0.345515       1        1  26.555182
8      0.114594 -0.417505       2        1  21.383314
18    -0.112161 -0.340646       3        1  12.637018
26    -0.071671 -0.449456       4        1  12.583918
0      0.102294 -0.458932       5        1  11.968214
12    -0.003067 -0.421579       6        1  10.656786
24    -0.217077 -0.344703       7        1   4.133389
27    -0.156567  0.112484       8        1   0.075786
15     0.013230  0.121118       9        1   0.000339
17     0.013233  0.121162      10        1   0.000313
2      0.013233  0.121163      11        1   0.000311
21     0.013234  0.121168      12        1   0.000307
5      0.013234  0.121170      13        1   0.000301
29     0.013234  0.121171      14        1   0.000300
10     0.013234  0.121173      15        1   0.000300
25     0.013234  0.121177      16        1   0.000298
16     0.013234  0.121176      17        1   0.000298
3      0.013234  0.121178      18        1   0.000293
9      0.013235  0.121179      19        1   0.000288
14     0.013235  0.121180      20        1   0.000285
6      0.013235  0.121181      21        1   0.000282
13     0.013235  0.121181      22        1   0.000282
11     0.013235  0.121182      23        1   0.000280
19     0.013235  0.121182      24        1   0.000278
4      0.013235  0.121183      25        1   0.000274
1      0.013235  0.121185      26        1   0.000273
23     0.013235  0.121186      27        1   0.000273
7      0.013235  0.121184      28        1   0.000273
28     0.013235  0.121186      29        1   0.000273
20     0.013235  0.121187      30        1   0.000272, topic_info=         Term          Freq         Total Category  logprob  loglift
682       run   2329.000000   2329.000000  Default  30.0000  30.0000
892      year   3484.000000   3484.000000  Default  29.0000  29.0000
343        go  24436.000000  24436.000000  Default  28.0000  28.0000
848      very  11462.000000  11462.000000  Default  27.0000  27.0000
734        so  20938.000000  20938.000000  Default  26.0000  26.0000
..        ...           ...           ...      ...      ...      ...
475      look      0.000165   3462.020094  Topic30  -9.6389  -4.0426
658  remember      0.000165   4083.568802  Topic30  -9.6389  -4.2077
433    jewish      0.000165   4670.651858  Topic30  -9.6390  -4.3421
884      work      0.000165   5756.971188  Topic30  -9.6390  -4.5512
522    mother      0.000165   3976.639986  Topic30  -9.6390  -4.1813

[3122 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
2909      2  0.968974  abortion
902       1  0.005456    access
902       3  0.092749    access
902       7  0.900208    access
7621      1  0.590982    accost
...     ...       ...       ...
892       7  0.036734      year
892       8  0.001722      year
3966      1  0.057433     zloty
3966      2  0.143582     zloty
3966      5  0.775344     zloty

[1907 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[23, 9, 19, 27, 1, 13, 25, 28, 16, 18, 3, 22, 6, 30, 11, 26, 17, 4, 10, 15, 7, 14, 12, 20, 5, 2, 24, 8, 29, 21])